#### 1.변수 설정

In [ ]:
search_key_excel = '조직역량개발그룹_23-1차 P직군 신입사원 도입교육 시간표(안) 외(운영자)'
search_key_ppt = '[23-1]포스코 p직군 신입사원 도입교육 강의의뢰서 모음'

save_path = 'D:\edu_schedule.xlsx'
excel_s_name = '23-1차 강사(연락처 외)'

mail_sub1 = '신입사원 교육중 '
mail_sub2 = ' 강의 리마인드 드립니다.'
mail_con = '''안녕하십니까?
            \n23-1차 포스코 P직군 신입사원 교육 강의 지원에 감사드립니다.
            \nP직군신입 담당자 황영미 입니다.
            \n강의 일정 리마인드 및 시험문제 출제안내 드립니다.
            \n •일정 : 2.9(목) 13:30~15:30 (2hr)
            \n•주제 : 포스코 제안활동
            \n•장소 : 인천 송도 포스코 인재창조원 러닝센터  201호(대강당)
            \n•시험문제 출제 : ~ 2.2(목)까지 메일전달 必  *중간평가 3주차(2.9) 진행예정
            \n사지선다 형식으로 2~3 문항 시험문제 출제 요청 (난이도 중상)
            \n늘 도움 주셔서 감사드립니다.
            \n관련하여 문의사항은 언제든 연락주세요\n\n'''

####  2.패키지 불러오기

In [ ]:
import pyautogui as pg
import clipboard as cb
from datetime import datetime

- *** 작업할 일자 입력 팝업창 ***

In [ ]:
edu_order_name = pg.prompt(title='작업할 일자 입력', text='메일을 보낼 일자를 입력하세요, 예:2023-01-26')

#### 3.EP 접속

- EP 로그인 정보 입력(ID, PASS)

In [ ]:
my_id = 'yooleenes'
my_pw = pg.password(title='EP_PASS', text='패스워드를 입력하세요', default='')

- EP 접속

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import clipboard as cb

url = 'http://swpsso.posco.net/idms/U61/jsp/login/login.jsp'

driver = webdriver.Edge('../msedgedriver')
driver.get(url)
driver.maximize_window()

#로그인
driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(my_id)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(my_pw)
driver.find_element(By.XPATH,'//*[@id="loginsubmit"]').click()
driver.implicitly_wait(10) #드라이버가 로딩되는데 10초까지 대기, 빨리 로딩되면 다음 코드 진행

# 로그인시 발생하는 예외 처리
pg.sleep(5)
try:  # 기존 로그인 경고창이 열릴 경우 alert accept 처리
    alert=driver.switch_to.alert
    alert.accept()
except:
    print("기존 로그인 경고창 없음. 계속 진행.")  # 경고창이 열리지 않을 경우

pg.sleep(15) 

#### 4.ECM 접속

- ECM 열기

In [ ]:
driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
driver.find_element(By.XPATH, '//*[@id="4"]/a[1]/span[1]/img').click()

- 일정표(엑셀) 검색

In [ ]:
pg.sleep(3)

driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
driver.find_element(By.XPATH, '//*[@id="searchValueT"]').send_keys(search_key_excel)
driver.find_element(By.XPATH, '//*[@id="searchValueT"]').send_keys(Keys.ENTER)


- 일정표(엑셀) 열기

In [ ]:
pg.sleep(3)

win = pg.getWindowsWithTitle('ECM')[0]
win.activate()

# aa = driver.find_element(By.CSS_SELECTOR, '#contentsList td:nth-child(1) a > img').click()
pg.press('tab', presses=32, interval=0.1)
pg.press('enter')

driver.find_element(By.XPATH, '//*[@id="A_CC_VIEW"]').click()
pg.sleep(15)

- 엑셀 다른이름으로 저장

In [ ]:
win = pg.getWindowsWithTitle('시간표')[0]
# win = pg.getWindowsWithTitle('edu')[0]
win.activate()

pg.press('f12')
pg.write(save_path, interval=0.2)
pg.hotkey('enter')
pg.sleep(0.5)

try:
      win_save = pg.locateCenterOnScreen('win_save.PNG')
      if win_save != None: pg.press('y')
except :
      print('다른 이름으로 저장 확인 창 없음, 계속 진행')

pg.sleep(2)
pg.hotkey('alt','f4')

- ppt 검색

In [ ]:
pg.sleep(3)

driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환

driver.find_element(By.XPATH, '//*[@id="searchValueT"]').clear()

driver.find_element(By.XPATH, '//*[@id="searchValueT"]').send_keys(search_key_ppt)
driver.find_element(By.XPATH, '//*[@id="searchValueT"]').send_keys(Keys.ENTER)


- ppt 열기

In [ ]:
pg.sleep(3)

win = pg.getWindowsWithTitle('ECM')[0]
win.activate()

# aa = driver.find_element(By.CSS_SELECTOR, '#contentsList td:nth-child(1) a > img').click()
pg.press('tab', presses=32, interval=0.1)
pg.press('enter')

driver.find_element(By.XPATH, '//*[@id="A_CC_VIEW"]').click()
pg.sleep(10)

#### 5.엑셀 일정표 데이터 추출

- 엑셀 일정표 열기

In [ ]:
import win32com.client
excel = win32com.client.Dispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Open(save_path) #정장 한 엑셀
ws = wb.Worksheets(excel_s_name)
pg.sleep(5)

- 행 카운트

In [ ]:
range_used = ws.UsedRange
max_row = range_used.Rows.Count

print(max_row)

- 작업해야 할 과목명 추출

In [ ]:
edu_name = []
edu_mail = []

start_row = 6

for r in range(start_row, max_row+1):
      cell_date = ws.Cells(r, 3).Value
      try: cell_date = datetime.strftime(cell_date, '%Y-%m-%d')
      except: print('일자 없음')
      if cell_date == edu_order_name:
            edu_name.append(ws.Cells(r, 2).Value) #과목명
            edu_mail.append(ws.Cells(r, 12).Value) #강사 메일 주소

#### 6.PPT 과목별 슬라이드 복사해서 메일 보내기

In [ ]:
#메일 화면 열기
for win in driver.window_handles:
      driver.switch_to.window(win)
      if driver.title == 'EP(Enterprise Portal)':
            driver.find_element(By.XPATH,"//*[@id='533982']").click()
pg.sleep(3)


for i in range(len(edu_name)): #과목명 수 만큼 반복
      win = pg.getWindowsWithTitle('강의의뢰서')[0]
      win.activate()

      cb.copy(edu_name[i])
      pg.hotkey('ctrl','f')
      pg.hotkey('ctrl','v')
      pg.press('enter')
      pg.sleep(1)
      pg.press('esc', presses=2, interval=0.5) #검색창 닫고, 표 선택

      # pg.hotkey('ctrl','a') #표만 선택
      pg.hotkey('ctrl','c')
      pg.sleep(1)

#메일 작성, 발송___________________________________________________________________________
      #메일 쓰기 화면 열기
      driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
      driver.find_element(By.XPATH,"//*[@id='Lnb']/div[1]/a").click()
      
      pg.sleep(1)

      #메일 쓰기
      driver.switch_to.window(driver.window_handles[-1]) #최근창으로 전환
      driver.find_element(By.XPATH,'//*[@id="token-input-send_to"]').send_keys(edu_mail[i])
      driver.find_element(By.XPATH,'//*[@id="token-input-send_to"]').send_keys(Keys.ENTER)

      pg.sleep(1)
      mail_sub = mail_sub1 + edu_name[i] + mail_sub2
      driver.find_element(By.XPATH,'//*[@id="write_send_info"]/table[2]/tbody/tr[1]/td/input').send_keys(mail_sub)

      #iframe으로 프레임 안에 프레임이 있음 : id값으로 프레임전환
      driver.switch_to.frame("dext_frame_WEC")
      driver.switch_to.frame("dext5_design_WEC")
      driver.find_element(By.XPATH,'//*[@id="dext_body"]/p[1]').send_keys(mail_con)

      #글꼴 편집
      win = pg.getWindowsWithTitle('메일쓰기')[0]
      win.activate()

      #표 붙이기
      pg.sleep(1)
      pg.hotkey('ctrl','v')
      pg.sleep(1)
      pg.press('tab', presses=5, interval=0.5)
      pg.write('900', interval=0.2)
      pg.sleep(1)
      pg.press('tab', presses=5, interval=0.5)
      pg.sleep(1)
      pg.press('enter')

      pg.sleep(1)

      pg.hotkey('ctrl','home')
      pg.sleep(1)
      pg.press('down',presses=6, interval=0.5)
      pg.sleep(1)
      pg.hotkey('shiftleft','shiftright','end')
      pg.sleep(1)
      pg.hotkey('ctrl','b')
      pg.sleep(1)

      #글꼴 색상 선택을 위한 프레임 이동
      driver.switch_to.default_content()
      driver.switch_to.frame("dext_frame_WEC")
      driver.find_element(By.XPATH,'//*[@id="ue_WECdetail_font_color"]').click()   

      pg.press('right',presses=33, interval=0.3)
      pg.press('enter')
      pg.sleep(1)

      #iframe 완전히 빠져 나가기
      driver.switch_to.default_content()
#____________________________________________________________________________
      pg.press('esc')
      pg.hotkey('ctrl','home') # 첫 슬라이드로 이동

      #메일 보내기
      pg.sleep(2)
      driver.find_element(By.XPATH,'//*[@id="memo_content"]/div[1]/ul[1]/li[1]/a').click()

      #메일 발송 확인창 확인 버튼 클릭- alert창으로 처리
      pg.sleep(5)
      try:  # 메일 발송 확인창이 열릴 경우 alert accept 처리
            alert=driver.switch_to.alert
            alert.accept()
      except:
            print("메일 발송 확인창 없음. 계속 진행.")

      pg.sleep(3)